# Qiskit Runtime

Qiskit Runtime is a new architecture offered by IBM Quantum that streamlines computations requiring many iterations. These experiments will execute significantly faster within this improved hybrid quantum/classical process.

Using Qiskit Runtime, for example, a research team at IBM Quantum was able to achieve 120x speed 
up in their lithium hydride simulation (link to come). 

Qiskit Runtime allows authorized users to upload their Qiskit quantum programs for themselves or 
others to use. A Qiskit quantum program, also called a Qiskit runtime program, is a piece of Python code that takes certain inputs, performs
quantum and maybe classical computation, and returns the processing results. The same or other
authorized users can then invoke these quantum programs by simply passing in the required input parameters.

<div class="alert alert-block alert-info">
<b>Note:</b> Qiskit Runtime is only available to select IBM Quantum providers. You can use the `has_service()` method to check if a provider has access:
</div>

In [1]:
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(project='qiskit-runtime')  # Change this to your provider.
can_use_runtime = provider.has_service('runtime')


If you don't have an IBM Quantum account, you can sign up for one on the [IBM Quantum](https://quantum-computing.ibm.com/) page.

## Listing programs <a name='listing_program'>

The `provider.runtime` object is an instance of the [`IBMRuntimeService`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService) class and serves as the main entry point to using the runtime service. It has three methods that can be used to find metadata of available programs:
- `pprint_programs()`: pretty prints metadata of all available programs
- `programs()`: returns a list of `RuntimeProgram` instances
- `program()`: returns a single `RuntimeProgram` instance

The metadata of a runtime program includes its ID, name, description, version, input parameters, return values, interim results, maximum execution time, and backend requirements. Maximum execution time is the maximum amount of time, in seconds, a program can run before being forcibly terminated.

To print the metadata of all available programs:

In [2]:
provider.runtime.pprint_programs()

circuit-runner:
  Name: circuit-runner
  Description: A runtime program that takes one or more circuits, compiles them, executes them, and optionally applies measurement error mitigation.
  Version: 1
  Creation date: 2021-05-07T00:17:07Z
  Max execution time: 1800
  Input parameters:
    - circuits:
      Description: A circuit or a list of circuits.
      Type: A QuantumCircuit or a list of QuantumCircuits.
      Required: True
    - shots:
      Description: Number of repetitions of each circuit, for sampling. Default: 1024.
      Type: int
      Required: False
    - initial_layout:
      Description: Initial position of virtual qubits on physical qubits.
      Type: dict or list
      Required: False
    - layout_method:
      Description: Name of layout selection pass ('trivial', 'dense', 'noise_adaptive', 'sabre')
      Type: string
      Required: False
    - routing_method:
      Description: Name of routing pass ('basic', 'lookahead', 'stochastic', 'sabre').
      Type: strin

To print the metadata of the program `sample-program`:

In [3]:
program = provider.runtime.program('sample-program')
print(program)

sample-program:
  Name: sample-program
  Description: A sample runtime program.
  Version: 1
  Creation date: 2021-06-04T14:11:19Z
  Max execution time: 300
  Input parameters:
    - iterations:
      Description: Number of iterations to run. Each iteration generates and runs a random circuit.
      Type: int
      Required: True
  Interim results:
    - iteration:
      Description: Iteration number.
      Type: int
    - counts:
      Description: Histogram data of the circuit result.
      Type: dict
  Returns:
    - -:
      Description: A string that says 'All done!'.
      Type: string


As you can see from above, the program `sample-program` is a simple program that has only 1 input parameter `iterations`, which indicates how many iterations to run. For each iteration it generates and runs a random 5-qubit circuit and returns the counts as well as the iteration number as the interim results. When the program finishes, it returns the sentence `All done!`. This program can only run for 300 seconds (5 minutes), and requires a backend that has at least 5 qubits.

## Invoking a runtime program <a name='invoking_program'>

You can use the [`IBMRuntimeService.run()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.run) method to invoke a runtime program. This method takes the following parameters:

- `program_id`: ID of the program to run
- `inputs`: Program input parameters. These input values are passed to the runtime program.
- `options`: Runtime options. These options control the execution environment. Currently the only available option is `backend_name`, which is required.
- `callback`: Callback function to be invoked for any interim results. The callback function will receive 2 positional parameters: job ID and interim result.
- `result_decoder`: Optional class used to decode job result.

Before we run a quantum program, we may want to define a callback function that would process interim results, which are intermediate data provided by a program while its still running. 

As we saw earlier, the metadata of `sample-program` says that its interim results are the iteration number and the counts of the randomly generated circuit. Here we define a simple callback function that just prints these interim results:

In [4]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}")

The following example runs the `sample-program` program with 3 iterations on `ibmq_montreal` and waits for its result. You can also use a different backend that supports Qiskit Runtime:

In [5]:
backend = provider.get_backend('ibmq_montreal')
program_inputs = {
    'iterations': 3
}
options = {'backend_name': backend.name()}
job = provider.runtime.run(program_id="sample-program",
                           options=options,
                           inputs=program_inputs,
                           callback=interim_result_callback
                          )
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

job id: c2ajo0m0lb0ph8orsprg
interim result: {'iteration': 0, 'counts': {'00000': 92, '00001': 15, '10000': 194, '10001': 21, '10010': 181, '10011': 33, '10100': 50, '10101': 9, '10110': 68, '10111': 12, '11000': 11, '11001': 5, '11010': 14, '11011': 5, '11100': 13, '11101': 2, '11110': 14, '11111': 4, '00010': 128, '00011': 16, '00100': 33, '00101': 11, '00110': 46, '00111': 9, '01000': 6, '01001': 5, '01010': 10, '01011': 3, '01100': 10, '01110': 3, '01111': 1}}
interim result: {'iteration': 1, 'counts': {'00000': 15, '00001': 9, '10000': 8, '10001': 9, '10010': 11, '10011': 6, '10100': 9, '10101': 9, '10110': 7, '10111': 3, '11000': 23, '11001': 26, '11010': 193, '11011': 113, '11100': 14, '11101': 13, '11110': 30, '11111': 25, '00010': 6, '00011': 4, '00100': 6, '00101': 2, '00110': 4, '00111': 3, '01000': 202, '01001': 97, '01010': 31, '01011': 23, '01100': 54, '01101': 45, '01110': 10, '01111': 14}}
interim result: {'iteration': 2, 'counts': {'00000': 33, '00001': 46, '10000': 27

The `run()` method returns a [`RuntimeJob`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.RuntimeJob.html#qiskit.providers.ibmq.runtime.RuntimeJob) instace, which is similar to the `Job` instance returned by regular `backend.run()`. `RuntimeJob` supports the following methods:

- `status()`: Return job status.
- `result()`: Wait for the job to finish and return the final result.
- `cancel()`: Cancel the job.
- `wait_for_final_state()`: Wait for the job to finish.
- `stream_results()`: Stream interim results. This can be used to start streaming the interim results if a `callback` function was not passed to the `run()` method.
- `job_id()`: Return the job ID.
- `backend()`: Return the backend where the job is run.

## Retrieving old jobs

You can use the [`IBMRuntimeService.job()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.job) method to retrieve a previously executed runtime job. Attributes of this [`RuntimeJob`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.RuntimeJob.html#qiskit.providers.ibmq.runtime.RuntimeJob) instace can tell you about the execution:

In [6]:
retrieved_job = provider.runtime.job(job.job_id())
print(f"Job {retrieved_job.job_id()} is an execution instance of runtime program {retrieved_job.program_id}.")
print(f"This job ran on backend {retrieved_job.backend()} and had input parameters {retrieved_job.inputs}")

Job c2ajo0m0lb0ph8orsprg is an execution instance of runtime program sample-program.
This job ran on backend ibmq_montreal and had input parameters {'iterations': 3}


Similarly, you can use [`IBMRuntimeService.jobs()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.jobs) to get a list of jobs. You can specify a limit on how many jobs to return. The default limit is 10:

In [7]:
retrieved_jobs = provider.runtime.jobs(limit=1)
for rjob in retrieved_jobs:
    print(rjob.job_id())

c2ajo0m0lb0ph8orsprg
c2ajjgm0lb0ph8orspdg


## Deleting a job

You can use the [`IBMRuntimeService.delete_job()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.delete_job) method to delete a job. You can only delete your own jobs, and this action cannot be reversed. 

In [8]:
provider.runtime.delete_job(job.job_id())

In [9]:
import qiskit.tools.jupyter
%qiskit_version_table